In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from mnist import MNIST
from sklearn.preprocessing import OneHotEncoder
#import tensorflow_datasets as tfds

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [2]:
def load_data():
    mndata = MNIST('./Data')
    train_X, train_Y = mndata.load_training()
    test_X, test_Y = mndata.load_testing()
    train_X =np.asarray(train_X)
    test_X = np.asarray(test_X)
    train_Y = np.asarray(train_Y)
    test_Y = np.asarray(test_Y)
    return train_X, train_Y, test_X, test_Y

In [3]:
def preprocess_image_data(image_data):
    """
    takes in an array object that is assumed to be X image data for MNIST.
    reshape to get grayscale 28x28 images for each row.
    converts array to float and normalizes values betweeen 0 and 1
    
    param image_data: a array.array object that is training or test data
    return image_array_norm: normalized image array
    """
    image_array = np.reshape(image_data, (image_data.shape[0], 28, 28, 1))
    image_array = image_array.astype(np.float32)
    image_array_norm = image_array / 255.0
    return image_array_norm

In [4]:
def preprocess_label_data(label_data):
    """
    takes in an array object and reshapes to 2D array. 
    One hot encodes labels since they are categorical.
    
    param label_data: label data
    return encoded_labels: -1 x 10 array of encoded data labels
    """
    label_array = label_data.reshape(-1, 1)
    hot_encoder = OneHotEncoder(dtype=np.uint8)
    hot_encoder.fit(label_array)
    encoded_labels = hot_encoder.transform(label_array).toarray()
    return encoded_labels

In [5]:
# load MNIST data set
Xtrain, Ytrain, Xtest, Ytest = load_data()

In [6]:
# Preprocess training and test data
Xtrain = preprocess_image_data(Xtrain)
Xtest = preprocess_image_data(Xtest)

In [7]:
# preprocess training and test labels 
Ytrain = preprocess_label_data(Ytrain)
Ytest = preprocess_label_data(Ytest)